In [1]:
import mysql.connector
from config import db_config
import pandas as pd
import requests
import json
import copy
import os

In [2]:
#dictionary of hockey team names, team ids, and team abbreviations
teams_dict = {
    'team_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 46, 47, 48, 49, 52, 53, 54, 55, 56],
    'team_name': ['New Jersey Devils', 'New York Islanders', 'New York Rangers', 'Philadelphia Flyers', 'Pittsburgh Penguins', 'Boston Bruins', 'Buffalo Sabres', 'Montreal Canadiens', 'Ottawa Senators', 'Toronto Maple Leafs', 'Atlanta Thrashers', 'Carolina Hurricanes', 'Florida Panthers', 'Tampa Bay Lightning', 'Washington Capitals', 'Chicago Blackhawks', 'Detroit Red Wings', 'Nashville Predators', 'St Louis Blues', 'Calgary Flames', 'Colorado Avalanche', 'Edmonton Oilers', 'Vancouver Canucks', 'Anaheim Ducks', 'Dallas Stars', 'Los Angeles Kings', 'Pheonix Coyotes', 'San Jose Sharks', 'Columbus Blue Jackets', 'Minnesota Wild', 'Minnesota North Stars', 'Quebec Nordique', 'Winnipeg Jets (1979)', 'Hartford Whalers', 'Colorado Rockies', 'Oakland Seals', 'Atlanta Flames', 'Kansas City Scouts', 'Cleveland Barons', 'Winnipeg Jets', 'Arizona Coyotes', 'Vegas Golden Knights', 'Seattle Kraken', 'California Golden Seals'],
    'team_abbreviation': ['NJD', 'NYI', 'NYR', 'PHI', 'PIT', 'BOS', 'BUF', 'MTL', 'OTT', 'TOR', 'ATL', 'CAR', 'FLA', 'TBL', 'WSH', 'CHI', 'DET', 'NSH', 'STL', 'CGY', 'COL', 'EDM', 'VAN', 'ANA', 'DAL', 'LAK', 'PHX', 'SJS', 'CBJ', 'MIN', 'MNS', 'QUE', 'WIN', 'HFD', 'CLR', 'OAK', 'AFM', 'KCS', 'CLE', 'WPG', 'ARI', 'VGK', 'SEA', 'CGS']
}

#Hartford Whalers are HFD not HAR

In [3]:
# Establish connection
connection = mysql.connector.connect(
    host=db_config["host"],
    user=db_config["user"],
    password=db_config["password"],
    database=db_config["database"]
)
cursor = connection.cursor()


# Create and populate games table

In [ ]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS games (
    game_id INT PRIMARY KEY,
    season_id INT,
    game_type INT,
    date DATE NULL,
    home_team_id INT,
    away_team_id INT,
    home_score INT,
    away_score INT,
    game_outcome VARCHAR(255),
    winning_goalie_id INT,
    winning_goal_scorer_id INT,
    series_status_round INT,
    FOREIGN KEY (home_team_id) REFERENCES teams(team_id),
    FOREIGN KEY (away_team_id) REFERENCES teams(team_id)
)
""")

connection.commit()
# game_id, season, gameType, gameDate, home_team, away_team, home_score, away_score, game_outcome

In [ ]:
# # Add foreign key constraints to the games table
# cursor.execute("""
# ALTER TABLE games
# ADD CONSTRAINT fk_home_team
# FOREIGN KEY (home_team_id) REFERENCES teams(team_id);
# """)

# cursor.execute("""
# ALTER TABLE games
# ADD CONSTRAINT fk_away_team
# FOREIGN KEY (away_team_id) REFERENCES teams(team_id);
# """)

# connection.commit()

In [ ]:
url_base = 'https://api-web.nhle.com/v1/club-schedule-season/'

#temp = teams_dict starting at team_abbreviation of CAR because they were the first to have a cancelled game
#temp = teams_dict['team_abbreviation'][11:]

#temp = teams_dict starting at team_abbreviation of EDM because they are missing a game date
#temp = teams_dict['team_abbreviation'][21:]


for abv in teams_dict['team_abbreviation']:
#for abv in temp:
                    #1942 start of original 6
    print(abv)
    games = []

    for year in range(1942, 2024):
        url = url_base + abv + '/' + str(year) + str(year + 1)
        response = requests.get(url)
        print(year)
        if response.text.strip():
            try:
                data = json.loads(response.text)
                if data['games']:
                    for g in data['games']:
                        #print(g)
                        game = (
                            g['id'],
                            g.get('season', ''),
                            g.get('gameType', -1),
                            g.get('gameDate', None),
                            g.get('homeTeam', {}).get('id', -1),
                            g.get('awayTeam', {}).get('id', -1),
                            g.get('homeTeam', {}).get('score', -1),
                            g.get('awayTeam', {}).get('score', -1),

                            g.get('gameOutcome', {}).get('lastPeriodType', ''),  # Use get method with default value ''
                            g.get('winningGoalie', {}).get('playerId', -1),  # Use get method with default value -1
                            g.get('winningGoalScorer', {}).get('playerId', -1),  # Use get method with default value -1
                            g.get('seriesStatus', {}).get('round', -1),  # Use get method with default value ''
                        )
                        #if homeTeamId or awayTeamId is not in teams_dict, skip it
                        if game[4] in teams_dict['team_id'] and game[5] in teams_dict['team_id']:
                            games.append(game)
            except json.JSONDecodeError:
                pass

    cursor.executemany("""
    INSERT IGNORE INTO games (game_id, season_id, game_type, date, home_team_id, away_team_id, home_score, away_score, game_outcome, winning_goalie_id, winning_goal_scorer_id, series_status_round)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, games)
    connection.commit()
    

NJD
1942
1943
1944
1945
1946
1947
1948
1949
1950


KeyboardInterrupt: 

# Create and populate teams table

In [16]:
# Create teams table
cursor.execute("""
CREATE TABLE IF NOT EXISTS teams (
    team_id INT PRIMARY KEY,
    team_name VARCHAR(255),
    team_abbreviation VARCHAR(10)
)
""")

# Populate teams table
teams_data = list(zip(teams_dict['team_id'], teams_dict['team_name'], teams_dict['team_abbreviation']))
cursor.executemany("""
INSERT IGNORE INTO teams (team_id, team_name, team_abbreviation)
VALUES (%s, %s, %s)
""", teams_data)

connection.commit()

# Create and populate a season table

In [ ]:
url_base = 'https://api-web.nhle.com/v1/standings/'
# Create seasons_end_standings table
cursor.execute("""
CREATE TABLE IF NOT EXISTS seasons_end_standings (
    season_id INT,
    team_id INT,
    wins INT,
    losses INT,
    ot_losses INT,
    points INT,
    games_played INT,
    goals_for INT,
    goals_against INT,
    preseason_wins INT,
    preseason_losses INT,
    preseason_ot_losses INT,
    preseason_goals_for INT,
    preseason_goals_against INT,
    preseason_games_played INT,
    playoff_wins INT,
    playoff_losses INT,
    playoff_ot_losses INT,
    playoff_goals_for INT,
    playoff_goals_against INT,
    playoff_games_played INT,
    conference_name VARCHAR(100),
    division_name VARCHAR(100),
    PRIMARY KEY (season_id, team_id),
    FOREIGN KEY (team_id) REFERENCES teams(team_id)
)
""")


# get data for each team for each season
# from the games table get: season, home_team_id, away_team_id, home_score, away_score, goals_for, goals_against
# for each season for each team, calculate wins, losses, ot_losses, points. Wins is when the team_id is either the home or away team and the score is greater than the other team's score. Losses is when the team_id is either the home or away team and the score is less than the other team's score. ot_losses is when the team_id is either the home or away team and the score is equal to the other team's score. Points is 2 * wins + ot_losses
# goals_for is the sum of the home_score when the team_id is the home team and the away_score when the team_id is the away team. goals_against is the sum of the away_score when the team_id is the home team and the home_score when the team_id is the away team

# Get data from games table
cursor.execute("""
SELECT season_id, home_team_id, away_team_id, home_score, away_score, game_outcome, game_type
FROM games
""")

games_data = cursor.fetchall()

for season in range(1942, 2025):
    season_id = int(str(season) + str(season + 1))
    seasons_end_standings_data = []

    url = url_base + str(season) + "-01-10"
    print(url)

    season_dict = {}

    response = requests.get(url)
    if response.text.strip():
        try:
            data = json.loads(response.text)
            if data['standings']:
                season_dict = {team['teamAbbrev']['default']: team for team in data['standings']}

        except json.JSONDecodeError:
            pass

    for team_id in teams_dict['team_id']:
        print(team_id)

        team_abbreviation = teams_dict['team_abbreviation'][teams_dict['team_id'].index(team_id)]

        #TODO flip team_id and season so that you can get the conference
        # and division names for the season then split them on team

        wins = 0
        preseason_wins = 0
        playoff_wins = 0
        losses = 0
        preseason_losses = 0
        playoff_losses = 0
        ot_losses = 0
        preseason_ot_losses = 0
        playoff_ot_losses = 0
        points = 0
        goals_for = 0
        preseason_goals_for = 0
        playoff_goals_for = 0
        goals_against = 0
        preseason_goals_against = 0
        playoff_goals_against = 0
        games_played = 0
        preseason_games_played = 0
        playoff_games_played = 0
        conference_name = ''
        division_name = ''

        if season_dict:
            team_data = season_dict.get(team_abbreviation, {})
            conference_name = team_data.get('conferenceName', '')
            division_name = team_data.get('divisionName', '')

        print(f"Season: {season_id}, Team: {team_id}, Abbreviation: {team_abbreviation}, Conference: {conference_name}, Division: {division_name}")

        for game in games_data:
            if game[0] == season_id:
                if game[1] == team_id:
                    print(game)
                    if game[6] == 1:
                        preseason_games_played += 1
                        preseason_goals_for += game[3]
                        preseason_goals_against += game[4]
                        if game[3] > game[4]:
                            preseason_wins += 1
                        elif game[3] < game[4]:
                            if game[5] == 'OT' or game[5] == 'SO':
                                preseason_ot_losses += 1
                            else:
                                preseason_losses += 1
                    elif game[6] == 2:
                        games_played += 1
                        goals_for += game[3]
                        goals_against += game[4]
                        if game[3] > game[4]:
                            wins += 1
                        elif game[3] < game[4]:
                            if game[5] == 'OT' or game[5] == 'SO':
                                ot_losses += 1
                            else:
                                losses += 1
                    elif game[6] == 3:
                        playoff_games_played += 1
                        playoff_goals_for += game[3]
                        playoff_goals_against += game[4]
                        if game[3] > game[4]:
                            playoff_wins += 1
                        elif game[3] < game[4]:
                            if game[5] == 'OT' or game[5] == 'SO':
                                playoff_ot_losses += 1
                            else:
                                playoff_losses += 1
                elif game[2] == team_id:
                    #print(game)
                    if game[6] == 1:
                        preseason_games_played += 1
                        preseason_goals_for += game[4]
                        preseason_goals_against += game[3]
                        if game[4] > game[3]:
                            preseason_wins += 1
                        elif game[4] < game[3]:
                            if game[5] == 'OT' or game[5] == 'SO':
                                preseason_ot_losses += 1
                            else:
                                preseason_losses += 1
                    elif game[6] == 2:
                        games_played += 1
                        goals_for += game[4]
                        goals_against += game[3]
                        if game[4] > game[3]:
                            wins += 1
                        elif game[4] < game[3]:
                            if game[5] == 'OT' or game[5] == 'SO':
                                ot_losses += 1
                            else:
                                losses += 1
                    elif game[6] == 3:
                        playoff_games_played += 1
                        playoff_goals_for += game[4]
                        playoff_goals_against += game[3]
                        if game[4] > game[3]:
                            playoff_wins += 1
                        elif game[4] < game[3]:
                            if game[5] == 'OT' or game[5] == 'SO':
                                playoff_ot_losses += 1
                            else:
                                playoff_losses += 1

        points = 2 * wins + ot_losses
        #print(f"Season_id: {season_id}, Team_id: {team_id}, Wins: {wins}, Losses: {losses}, OT_Losses: {ot_losses}, Points: {points}, Goals_For: {goals_for}, Goals_Against: {goals_against}")
        seasons_end_standings_data.append((season_id, team_id, wins, losses, ot_losses, points, games_played, goals_for, goals_against, preseason_wins, preseason_losses, preseason_ot_losses, preseason_goals_for, preseason_goals_against, preseason_games_played, playoff_wins, playoff_losses, playoff_ot_losses, playoff_goals_for, playoff_goals_against, playoff_games_played, conference_name, division_name))


    print(f"Standings: {seasons_end_standings_data}")
    # Populate seasons_end_standings table
    cursor.executemany("""
    INSERT IGNORE INTO seasons_end_standings (season_id, team_id, wins, losses, ot_losses, points, games_played, goals_for, goals_against, preseason_wins, preseason_losses, preseason_ot_losses, preseason_goals_for, preseason_goals_against, preseason_games_played, playoff_wins, playoff_losses, playoff_ot_losses, playoff_goals_for, playoff_goals_against, playoff_games_played, conference_name, division_name)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, seasons_end_standings_data)

    connection.commit()


https://api-web.nhle.com/v1/standings/1942-01-10
1
Season: 19421943, Team: 1, Abbreviation: NJD, Conference: , Division: 
2
Season: 19421943, Team: 2, Abbreviation: NYI, Conference: , Division: 
3
Season: 19421943, Team: 3, Abbreviation: NYR, Conference: , Division: 
(19421943, 3, 8, 4, 3, 'OT', 2)
(19421943, 3, 16, 5, 3, 'OT', 2)
(19421943, 3, 6, 3, 4, 'REG', 2)
(19421943, 3, 10, 3, 7, 'REG', 2)
(19421943, 3, 17, 4, 4, 'REG', 2)
(19421943, 3, 6, 3, 2, 'REG', 2)
(19421943, 3, 8, 3, 7, 'REG', 2)
(19421943, 3, 6, 3, 7, 'REG', 2)
(19421943, 3, 10, 2, 8, 'REG', 2)
(19421943, 3, 10, 3, 1, 'REG', 2)
(19421943, 3, 17, 0, 2, 'REG', 2)
(19421943, 3, 16, 3, 3, 'REG', 2)
(19421943, 3, 8, 4, 7, 'REG', 2)
(19421943, 3, 17, 1, 4, 'REG', 2)
(19421943, 3, 6, 3, 6, 'REG', 2)
(19421943, 3, 6, 2, 7, 'REG', 2)
(19421943, 3, 16, 1, 1, 'REG', 2)
(19421943, 3, 10, 4, 4, 'REG', 2)
(19421943, 3, 17, 4, 5, 'REG', 2)
(19421943, 3, 8, 6, 1, 'REG', 2)
(19421943, 3, 16, 7, 4, 'REG', 2)
(19421943, 3, 17, 1, 5, 'REG'

# Create and populate a game event table

In [6]:
url_base = "https://api-web.nhle.com/v1/gamecenter/"#2023020204/play-by-play

# Create the plays table if it doesn't exist
cursor.execute("""
CREATE TABLE IF NOT EXISTS events (
    event_id INT,
    game_id BIGINT,
    period_number INT,
    period_type VARCHAR(10),
    time_in_period VARCHAR(10),
    time_remaining VARCHAR(10),
    situation_code VARCHAR(10),
    home_team_defending_side VARCHAR(10),
    type_code INT,
    type_desc_key VARCHAR(50),
    sort_order INT,
    x_coord INT,
    y_coord INT,
    zone_code VARCHAR(5),
    shot_type VARCHAR(20),
    blocking_Player_id INT,
    shooting_player_id INT,
    goalie_in_net_id INT,
    player_id INT,
    event_owner_team_id INT,
    away_sog INT,
    home_sog INT,
    hitting_player_id INT,
    hittee_player_id INT,
    reason VARCHAR(20),
    secondary_reason VARCHAR(20),
    losing_player_id INT,
    winning_player_id INT,
    PRIMARY KEY (event_id, game_id)
)
""")
connection.commit()

#grab each game_id from the games table
cursor.execute("""
SELECT DISTINCT game_id FROM games
""")

game_ids = cursor.fetchall()

cursor.execute("""
SELECT DISTINCT game_id from events 
""")

done_game_ids = cursor.fetchall()
# Create a set of game IDs that have already been processed

for id in game_ids:

    # Check if the game ID is already in the done_game_ids set
    if id[0] in done_game_ids:
        print(f"Game ID {id[0]} already processed. Skipping.")
        continue
    else:
        #add the game ID to the done_game_ids set
        done_game_ids.append(id)

    #if done_game_ids is divisible by 100, print the number of game_ids processed
    if len(done_game_ids) % 100 == 0:
        print(f"Processed {len(done_game_ids)} out of {len(game_ids)} game IDs.")

    url = url_base + str(id[0]) + '/play-by-play'
    #print(url)
    response = requests.get(url)

    if response.text.strip():
        try:
            data = json.loads(response.text)
            #print(data)
            if 'plays' in data:
                plays_data = []
                for play in data['plays']:
                    # Extract play data
                    periodDescriptor = play.get('periodDescriptor', {})
                    details = play.get('details', {})
                    #print(play['typeDescKey'])
                    #print(details)
                    play_data = (
                        play.get('eventId', -1),
                        id[0],
                        periodDescriptor.get('number', -1),
                        periodDescriptor.get('periodType', ''),
                        play.get('timeInPeriod', ''),
                        play.get('timeRemaining', ''),
                        play.get('situationCode', -1),
                        play.get('homeTeamDefendingSide', ''),
                        play.get('typeCode', -1),
                        play.get('typeDescKey', ''),
                        play.get('sortOrder', -1),
                        details.get('xCoord', -1),
                        details.get('yCoord', -1),
                        details.get('zoneCode', ''),
                        details.get('shotType', ''),
                        details.get('blockingPlayerId', -1),
                        details.get('shootingPlayerId', -1),
                        details.get('goalieInNetId', -1),
                        details.get('playerId', -1),
                        details.get('eventOwnerTeamId', -1),
                        details.get('awaySOG', -1),
                        details.get('homeSOG', -1),
                        details.get('hittingPlayerId', -1),
                        details.get('hitteePlayerId', -1),
                        details.get('reason', ''),
                        details.get('secondaryReason', ''),
                        details.get('losingPlayerId', -1),
                        details.get('winningPlayerId', -1)
                    )
                    plays_data.append(play_data)


                # Insert play data into the database
                cursor.executemany("""
                INSERT IGNORE INTO events (
                    event_id, game_id, period_number, period_type, time_in_period, time_remaining, situation_code,
                    home_team_defending_side, type_code, type_desc_key, sort_order, x_coord, y_coord, zone_code,
                    shot_type, blocking_player_id, shooting_player_id, goalie_in_net_id, player_id, event_owner_team_id,
                    away_sog, home_sog, hitting_player_id, hittee_player_id, reason, secondary_reason,
                    losing_player_id, winning_player_id
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, plays_data)
                connection.commit()
        except json.JSONDecodeError:
            pass

Processed 64800 out of 65293 game IDs.
Processed 64900 out of 65293 game IDs.
Processed 65000 out of 65293 game IDs.
Processed 65100 out of 65293 game IDs.
Processed 65200 out of 65293 game IDs.
Processed 65300 out of 65293 game IDs.
Processed 65400 out of 65293 game IDs.


KeyboardInterrupt: 

In [10]:
#check which games are missing from the events table
cursor.execute("""
SELECT game_id FROM games
""")
games = cursor.fetchall()

cursor.execute("""
SELECT DISTINCT game_id FROM events
""")
events = cursor.fetchall()

missing_games = []
for game in games:
    if game not in events:
        missing_games.append(game[0])

print(missing_games)

[2005010063, 2005010070, 2005010092, 2006010001, 2006010011, 2006010082, 2007010011, 2007010079, 2008010021, 2008010064, 2008010087, 2008010095, 2011010022, 2005010018, 2005010050, 2006010035, 2006010044, 2006010085, 2006010096, 2007010012, 2007010022, 2007010024, 2007010063, 2007010102, 2008010011, 2008010054, 2008010076, 2009010040, 2009010049, 2010010064, 2010010092, 2011010097, 2013010026, 2013010053, 2013010103, 2014010011, 2014010012, 2014010094, 2016010052, 2017010003, 2017010044, 1959010202, 1959010203, 1959010205, 1959010208, 1959010209, 1959010210, 1959010212, 1959010215, 1959010216, 1959010217, 1959010218, 1959010219, 1993010201, 2005010056, 2005010083, 2005010107, 2006010020, 2006010045, 2006010064, 2006010070, 2007010044, 2007010048, 2007010091, 2008010007, 2008010034, 2008010049, 2005010027, 2005010048, 2005010067, 2005010084, 2006010027, 2006010047, 2006010068, 2007010020, 2007010061, 2007010075, 2007010097, 2008010006, 2008010035, 2008010045, 2008010078, 2009010022, 201

In [ ]:
url_base = "https://api-web.nhle.com/v1/gamecenter/"#2023020204/play-by-play

count = 0

for id in missing_games:
    count += 1
    #if done_game_ids is divisible by 100, print the number of game_ids processed
    if count % 100 == 0:
        print(f"Processed {count} out of {len(missing_games)} game IDs.")

    url = url_base + str(id) + '/play-by-play'
    #print(url)
    response = requests.get(url)

    if response.text.strip():
        try:
            data = json.loads(response.text)
            #print(data)
            if 'plays' in data:
                plays_data = []
                for play in data['plays']:
                    # Extract play data
                    periodDescriptor = play.get('periodDescriptor', {})
                    details = play.get('details', {})
                    #print(play['typeDescKey'])
                    #print(details)
                    play_data = (
                        play.get('eventId', -1),
                        id,
                        periodDescriptor.get('number', -1),
                        periodDescriptor.get('periodType', ''),
                        play.get('timeInPeriod', ''),
                        play.get('timeRemaining', ''),
                        play.get('situationCode', -1),
                        play.get('homeTeamDefendingSide', ''),
                        play.get('typeCode', -1),
                        play.get('typeDescKey', ''),
                        play.get('sortOrder', -1),
                        details.get('xCoord', -1),
                        details.get('yCoord', -1),
                        details.get('zoneCode', ''),
                        details.get('shotType', ''),
                        details.get('blockingPlayerId', -1),
                        details.get('shootingPlayerId', -1),
                        details.get('goalieInNetId', -1),
                        details.get('playerId', -1),
                        details.get('eventOwnerTeamId', -1),
                        details.get('awaySOG', -1),
                        details.get('homeSOG', -1),
                        details.get('hittingPlayerId', -1),
                        details.get('hitteePlayerId', -1),
                        details.get('reason', ''),
                        details.get('secondaryReason', ''),
                        details.get('losingPlayerId', -1),
                        details.get('winningPlayerId', -1)
                    )
                    plays_data.append(play_data)


                # Insert play data into the database
                cursor.executemany("""
                INSERT IGNORE INTO events (
                    event_id, game_id, period_number, period_type, time_in_period, time_remaining, situation_code,
                    home_team_defending_side, type_code, type_desc_key, sort_order, x_coord, y_coord, zone_code,
                    shot_type, blocking_player_id, shooting_player_id, goalie_in_net_id, player_id, event_owner_team_id,
                    away_sog, home_sog, hitting_player_id, hittee_player_id, reason, secondary_reason,
                    losing_player_id, winning_player_id
                )
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, plays_data)
                connection.commit()
        except json.JSONDecodeError:
            print(f"JSONDecodeError for game ID {id}.")
            pass

Processed 100 out of 565 game IDs.
Processed 200 out of 565 game IDs.
Processed 300 out of 565 game IDs.
Processed 400 out of 565 game IDs.
Processed 500 out of 565 game IDs.


# Create and populate a team roster table

For the roster and players:
roster gives : {
			"id": 8476805,
			"headshot": "https://assets.nhle.com/mugs/nhl/20132014/TOR/8476805.png",
			"firstName": {
				"default": "Spencer"
			},
			"lastName": {
				"default": "Abbott"
			},
			"sweaterNumber": 56,
			"positionCode": "L",
			"shootsCatches": "R",
			"heightInInches": 69,
			"weightInPounds": 170,
			"heightInCentimeters": 175,
			"weightInKilograms": 77,
			"birthDate": "1988-04-30",
			"birthCity": {
				"default": "Hamilton"
			},
			"birthCountry": "CAN",
			"birthStateProvince": {
				"default": "ON"
			}

player gives: 		{
			"assists": 29,
			"evGoals": 14,
			"evPoints": 34,
			"faceoffWinPct": 0.52671,
			"gameWinningGoals": 1,
			"gamesPlayed": 73,
			"goals": 18,
			"lastName": "Bozak",
			"otGoals": 0,
			"penaltyMinutes": 22,
			"playerId": 8475098,
			"plusMinus": -7,
			"points": 47,
			"pointsPerGame": 0.64383,
			"positionCode": "C",
			"ppGoals": 4,
			"ppPoints": 12,
			"seasonId": 20112012,
			"shGoals": 0,
			"shPoints": 1,
			"shootingPct": 0.16513,
			"shootsCatches": "R",
			"shots": 109,
			"skaterFullName": "Tyler Bozak",
			"teamAbbrevs": "TOR",
			"timeOnIcePerGame": 1130.5205
		},

-----------------------------------------------------------------

Player should have seasons with their stats from the season

Roster should probably just have a list of player ids per season

How is multiple teams for a player being handled?

In [49]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS roster_players (
    player_id INT,
    team_id INT,
    season_id INT,
    firstName VARCHAR(255),
    lastName VARCHAR(255),
    sweaterNumber INT,
	positionCode VARCHAR(10),
    shootsCatches VARCHAR(5),
    heightInInches INT,
    weightInPounds INT,
    heightInCentimeters INT,
    weightInKilograms INT,
    birthDate DATE,
    birthCity VARCHAR(255),
    birthCountry VARCHAR(255),
    -- FOREIGN KEY (player_id) REFERENCES players(player_id),
    FOREIGN KEY (team_id) REFERENCES teams(team_id),
    PRIMARY KEY (player_id, team_id, season_id)
)
""")
connection.commit()

seasons_end_standings_data = []

#grab each game_id from the games table
cursor.execute("""
SELECT DISTINCT season_id, team_id FROM seasons_end_standings WHERE games_played > 0
""")

season_id_team_id = cursor.fetchall()

for st in season_id_team_id:
    season_id = st[0]
    team_id = st[1]

    team_abv = teams_dict['team_abbreviation'][teams_dict['team_id'].index(team_id)]

    print(f"Season ID: {season_id}, Team ID: {team_abv}")

    url = f"https://api-web.nhle.com/v1/roster/{team_abv}/{season_id}"
    #print(url)
    response = requests.get(url)

    if response.text.strip():
        try:
            data = json.loads(response.text)
            #print(data)
            roster_data = []
            if 'forwards' in data:
                for player in data['forwards']:
                    # Extract player data
                    player_data = (
                        player.get('id', -1),
                        team_id,
                        season_id,
                        player.get('firstName', {}).get('default', ''),
                        player.get('lastName', {}).get('default', ''),
                        player.get('sweaterNumber', -1),
                        player.get('positionCode', ''),
                        player.get('shootsCatches', ''),
                        player.get('heightInInches', -1),
                        player.get('weightInPounds', -1),
                        player.get('heightInCentimeters', -1),
                        player.get('weightInKilograms', -1),
                        player.get('birthDate', ''),
                        player.get('birthCity', {}).get('default', ''),
                        player.get('birthCountry', '')
                    )
                    roster_data.append(player_data)

            if 'defensemen' in data:
                for player in data['defensemen']:
                    # Extract player data
                    player_data = (
                        player.get('id', -1),
                        team_id,
                        season_id,
                        player.get('firstName', {}).get('default', ''),
                        player.get('lastName', {}).get('default', ''),
                        player.get('sweaterNumber', -1),
                        player.get('positionCode', ''),
                        player.get('shootsCatches', ''),
                        player.get('heightInInches', -1),
                        player.get('weightInPounds', -1),
                        player.get('heightInCentimeters', -1),
                        player.get('weightInKilograms', -1),
                        player.get('birthDate', ''),
                        player.get('birthCity', {}).get('default', ''),
                        player.get('birthCountry', '')
                    )
                    roster_data.append(player_data)

            if 'goalies' in data:
                for player in data['goalies']:
                    # Extract player data
                    player_data = (
                        player.get('id', -1),
                        team_id,
                        season_id,
                        player.get('firstName', {}).get('default', ''),
                        player.get('lastName', {}).get('default', ''),
                        player.get('sweaterNumber', -1),
                        player.get('positionCode', ''),
                        player.get('shootsCatches', ''),
                        player.get('heightInInches', -1),
                        player.get('weightInPounds', -1),
                        player.get('heightInCentimeters', -1),
                        player.get('weightInKilograms', -1),
                        player.get('birthDate', ''),
                        player.get('birthCity', {}).get('default', ''),
                        player.get('birthCountry', '')
                    )
                    roster_data.append(player_data)

            #print(roster_data)

            # Insert roster data into the database
            cursor.executemany("""
            INSERT IGNORE INTO roster_players (
                player_id, team_id, season_id, firstName, lastName, sweaterNumber, positionCode,
                shootsCatches, heightInInches, weightInPounds, heightInCentimeters,
                weightInKilograms, birthDate, birthCity, birthCountry
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """, roster_data)
            connection.commit()

            #print("Rows inserted:", cursor.rowcount)

        except json.JSONDecodeError:
            print(f"JSONDecodeError for season ID {season_id} and team ID {team_abv}.")
            pass

    

Season ID: 19421943, Team ID: NYR
Season ID: 19421943, Team ID: BOS
Season ID: 19421943, Team ID: MTL
Season ID: 19421943, Team ID: TOR
Season ID: 19421943, Team ID: CHI
Season ID: 19421943, Team ID: DET
Season ID: 19431944, Team ID: NYR
Season ID: 19431944, Team ID: BOS
Season ID: 19431944, Team ID: MTL
Season ID: 19431944, Team ID: TOR
Season ID: 19431944, Team ID: CHI
Season ID: 19431944, Team ID: DET
Season ID: 19441945, Team ID: NYR
Season ID: 19441945, Team ID: BOS
Season ID: 19441945, Team ID: MTL
Season ID: 19441945, Team ID: TOR
Season ID: 19441945, Team ID: CHI
Season ID: 19441945, Team ID: DET
Season ID: 19451946, Team ID: NYR
Season ID: 19451946, Team ID: BOS
Season ID: 19451946, Team ID: MTL
Season ID: 19451946, Team ID: TOR
Season ID: 19451946, Team ID: CHI
Season ID: 19451946, Team ID: DET
Season ID: 19461947, Team ID: NYR
Season ID: 19461947, Team ID: BOS
Season ID: 19461947, Team ID: MTL
Season ID: 19461947, Team ID: TOR
Season ID: 19461947, Team ID: CHI
Season ID: 194

# Create and populate a player table

In [ ]:
# Create players_season table
cursor.execute("""
CREATE TABLE IF NOT EXISTS players_season (
    assists INT,
    ev_goals INT,
    ev_points INT,
    faceoff_win_pct FLOAT,
    game_winning_goals INT,
    games_played INT,
    goals INT,
    last_name VARCHAR(255),
    ot_goals INT,
    penalty_minutes INT,
    player_id INT,
    plus_minus INT,
    points INT,
    points_per_game FLOAT,
    position_code VARCHAR(10),
    pp_goals INT,
    pp_points INT,
    season_id INT,
    sh_goals INT,
    sh_points INT,
    shooting_percentage FLOAT,
    shoots_catches VARCHAR(5),
    shots INT,
    skater_full_name VARCHAR(255),
    team_abbreviations VARCHAR(10),
    time_on_ice_per_game FLOAT,
    PRIMARY KEY (player_id, season_id),
    FOREIGN KEY (player_id) REFERENCES roster_players(player_id)
)
""")
connection.commit()

url_base = "https://api.nhle.com/stats/rest/en/skater/summary?cayenneExp=playerId="#8477492

cursor.execute("""
SELECT DISTINCT player_id FROM roster_players;
""")
player_ids = cursor.fetchall()

for p_id in player_ids:
    player_id = p_id[0]
    url = url_base + str(player_id)
    response = requests.get(url)
    print(player_id)
    if response.text.strip():
        try:
            data = json.loads(response.text)
            if data['data']:
                for season in data['data']:
                    # Extract player data

                    player = (
                        season.get('assists', -1),
                        season.get('evGoals', -1),
                        season.get('evPoints', -1),
                        season.get('faceoffWinPct', -1),
                        season.get('gameWinningGoals', -1),
                        season.get('gamesPlayed', -1),
                        season.get('goals', -1),
                        season.get('lastName', ''),
                        season.get('otGoals', -1),
                        season.get('penaltyMinutes', -1),
                        player_id,
                        season.get('plusMinus', -1),
                        season.get('points', -1),
                        season.get('pointsPerGame', -1),
                        season.get('positionCode', ''),
                        season.get('ppGoals', -1),
                        season.get('ppPoints', -1),
                        season.get('seasonId', -1),
                        season.get('shGoals', -1),
                        season.get('shPoints', -1),
                        season.get('shootingPct', -1),
                        season.get('shootsCatches', ''),
                        season.get('shots', -1),
                        season.get('skaterFullName', ''),
                        season.get('teamAbbrevs', ''),
                        season.get('timeOnIcePerGame', -1)

                    )

                    cursor.execute("""
                        INSERT IGNORE INTO players_season (
                            assists, ev_goals, ev_points, faceoff_win_pct, game_winning_goals, games_played, goals, last_name, ot_goals, penalty_minutes, player_id, plus_minus, points, points_per_game, position_code, pp_goals, pp_points, season_id, sh_goals, sh_points, shooting_percentage, shoots_catches, shots, skater_full_name, team_abbreviations, time_on_ice_per_game
                        )  
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """, player)
                    connection.commit()

                    #print("Rows inserted:", cursor.rowcount)

        except json.JSONDecodeError:
            pass


8444853
8444854
8444856
8444857
8444858
8444859
8444860
8444865
8444866
8444867
8444868
8444869
8444871
8444872
8444873
8444876
8444877
8444878
8444879
8444882
8444890
8444892
8444893
8444894
8444896
8444897
8444898
8444899
8444905
8444908
8444912
8444913
8444914
8444917
8444918
8444919
8444923
8444924
8444926
8444928
8444929
8444933
8444934
8444943
8444944
8444945
8444947
8444952
8444953
8444954
8444955
8444957
8444961
8444962
8444963
8444966
8444969
8444970
8444971
8444972
8444973
8444979
8444980
8444981
8444983
8444984
8444985
8444986
8444988
8444992
8444993
8444995
8444996
8444997
8444999
8445000
8445001
8445002
8445003
8445004
8445005
8445006
8445007
8445009
8445013
8445015
8445017
8445018
8445019
8445020
8445021
8445022
8445024
8445025
8445027
8445028
8445032
8445033
8445034
8445035
8445036
8445038
8445039
8445040
8445041
8445043
8445045
8445046
8445051
8445053
8445054
8445055
8445056
8445058
8445059
8445062
8445063
8445064
8445065
8445067
8445069
8445070
8445072
8445073
8445078


In [82]:
#create players table
cursor.execute("""
CREATE TABLE IF NOT EXISTS players (
    player_id INT PRIMARY KEY,
    skaterFullName VARCHAR(255),
    birth_date DATE,
    birth_city VARCHAR(255),
    birth_country VARCHAR(255),
    positionCode VARCHAR(10),
    shootsCatches VARCHAR(5),
    games_played INT,
    total_assists INT,
    total_goals INT,
    total_points INT,
    total_penalty_minutes INT,
    total_games_played INT,
    total_plus_minus INT
)
""")

connection.commit()

cursor.execute("""
SELECT player_id, MAX(skater_full_name), MAX(position_code), MAX(shoots_catches), SUM(assists),
    SUM(goals), SUM(points), SUM(penalty_minutes), SUM(games_played), SUM(plus_minus)
    FROM players_season
    GROUP BY player_id;
""")

players = cursor.fetchall()

for player in players:
    print(player[0])
    player_id = player[0]
    skater_full_name = player[1]
    position_code = player[2]
    shoots_catches = player[3]
    total_assists = player[4]
    total_goals = player[5]
    total_points = player[6]
    total_penalty_minutes = player[7]
    total_games_played = player[8]
    total_plus_minus = player[9]

    # Get birth date, city, and country from roster_players table
    cursor.execute("""
        SELECT birthDate, birthCity, birthCountry
        FROM roster_players
        WHERE player_id = %s
        LIMIT 1;
    """, (player_id,))
    
    birth_data = cursor.fetchone()
    
    if birth_data:
        birth_date, birth_city, birth_country = birth_data
    else:
        birth_date, birth_city, birth_country = None, None, None

    # Insert or update the player in the players table
    cursor.execute("""
        INSERT IGNORE INTO players (player_id, skaterFullName, birth_date, birth_city, birth_country,
            positionCode, shootsCatches, games_played, total_assists, total_goals,
            total_points, total_penalty_minutes, total_games_played, total_plus_minus)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (player_id, skater_full_name, birth_date, birth_city, birth_country, position_code, shoots_catches, total_games_played, total_assists, total_goals, total_points, total_penalty_minutes, total_games_played, total_plus_minus))
    connection.commit()
    #print("Rows inserted:", cursor.rowcount)







8444853
8444854
8444856
8444857
8444858
8444859
8444860
8444865
8444866
8444867
8444868
8444869
8444871
8444872
8444873
8444876
8444877
8444878
8444879
8444882
8444890
8444892
8444893
8444894
8444896
8444897
8444898
8444899
8444905
8444908
8444912
8444913
8444914
8444917
8444918
8444919
8444923
8444924
8444926
8444928
8444929
8444933
8444934
8444943
8444944
8444945
8444947
8444952
8444953
8444954
8444955
8444957
8444961
8444962
8444963
8444966
8444969
8444970
8444971
8444972
8444973
8444979
8444980
8444981
8444983
8444984
8444985
8444986
8444988
8444992
8444993
8444995
8444996
8444997
8444999
8445000
8445001
8445002
8445003
8445004
8445005
8445006
8445007
8445009
8445013
8445015
8445017
8445018
8445019
8445020
8445021
8445022
8445024
8445025
8445027
8445028
8445032
8445033
8445034
8445035
8445036
8445038
8445039
8445040
8445041
8445043
8445045
8445046
8445051
8445053
8445054
8445055
8445056
8445058
8445059
8445062
8445063
8445064
8445065
8445067
8445069
8445070
8445072
8445073
8445078


In [83]:
#query each table to see what columns are in each table
table_names = ["games", "teams", "seasons_end_standings", "events", "roster_players", "players_season", "players"]

for table in table_names:
    print(f"Columns in {table}:")
    cursor.execute(f"SHOW COLUMNS FROM {table}")
    columns = cursor.fetchall()
    for column in columns:
        print(column)
    print("-" * 40)



Columns in games:
('game_id', 'int', 'NO', 'PRI', None, '')
('season', 'int', 'YES', '', None, '')
('game_type', 'int', 'YES', '', None, '')
('date', 'date', 'YES', '', None, '')
('home_team_id', 'int', 'YES', 'MUL', None, '')
('away_team_id', 'int', 'YES', 'MUL', None, '')
('home_score', 'int', 'YES', '', None, '')
('away_score', 'int', 'YES', '', None, '')
('game_outcome', 'varchar(255)', 'YES', '', None, '')
('winning_goalie_id', 'int', 'YES', '', None, '')
('winning_goal_scorer_id', 'int', 'YES', '', None, '')
('series_status_round', 'int', 'YES', '', None, '')
----------------------------------------
Columns in teams:
('team_id', 'int', 'NO', 'PRI', None, '')
('team_name', 'varchar(255)', 'YES', '', None, '')
('team_abbreviation', 'varchar(10)', 'YES', '', None, '')
----------------------------------------
Columns in seasons_end_standings:
('season_id', 'int', 'NO', 'PRI', None, '')
('team_id', 'int', 'NO', 'PRI', None, '')
('wins', 'int', 'YES', '', None, '')
('losses', 'int', 'Y

In [ ]:
#adding a season lookup table
cursor.execute("""
CREATE TABLE IF NOT EXISTS seasons (
    season_id INT PRIMARY KEY,
    start_year INT,
    end_year INT
)
""")

# Populate seasons table with data from 1942 to 2024
for year in range(1942, 2024):
    season_id = int(str(year) + str(year + 1))
    cursor.execute("""
    INSERT IGNORE INTO seasons (season_id, start_year, end_year)
    VALUES (%s, %s, %s)
    """, (season_id, year, year + 1))

connection.commit()

# Adding foreign key constraints to the games table
cursor.execute("""
ALTER TABLE games
ADD CONSTRAINT fk_season_id
FOREIGN KEY (season_id) REFERENCES seasons(season_id)
""")

connection.commit()

NameError: name 'cursor' is not defined